### **Obtener noticias a través de Google**
_Por Juan Carlos Rodríguez-Raga y Andrés Mauricio Toloza Cruz_

Este código proporciona herramientas a través de Selenium y Newspaper3k para hacer web scrapping a noticias. 

##### **Primer paso: descargar e importar nuestras librerias**
Las librerias que estaremos usando son Selenium y Newspaper3k  para descargarlas usamos el siguiente código:

    ! pip install selenium
    ! pip install newspaper3k
    ! pip install lxml[html_clean]
    ! pip install pandas

Tambien usaremos datetime, time y urllib, pero esas ya vienen en nuestra instalación de Python

In [ ]:
! pip install pandas

In [ ]:
# Importamos las librerias necesarias

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium import webdriver
import time

import newspaper
from newspaper import ArticleException

from urllib.parse import urlparse
import json

import pandas as pd

from datetime import datetime

##### **Segundo paso: extraer las noticias**
En este caso, vamos a extraer las noticias del último mes de la sección de "Política" de los siguientes medios:
1. El Espectador
2. Revista Semana
3. La Silla Vacía
4. Noticias Caracol
5. El País Colombia

Debido a que cada página tiene diferentes estrucutras, hacemos una función para cada sitio web

In [6]:
# Función scroll para cargar toda la web

def scroll():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

In [25]:
# Función para obtener las noticias

def get_news(url, parse_data):
	
    try: 

        article = newspaper.Article(url=url, language='es')
        article.download()
        article.parse()

        article_dict ={
            "title": str(article.title),
            "published_date": str(article.publish_date),
            "top_image": str(article.top_image),
            "link": urlparse(url).netloc,
            "full_link": str(url),
            "text": str(article.text)
        }

        parse_data[article_dict["title"]] = article_dict

        time.sleep(5)

    except ArticleException:
        article_dict ={
            "title": "Error",
            "published_date": "Error",
            "top_image": "Error",
            "link": urlparse(url).netloc,
            "full_link": str(url),
            "text": "Error"
            }

        parse_data[article_dict["title"]] = article_dict

            

        time.sleep(5)

In [42]:
# Función para ajustar el date 
def adjust_date(parse_data):
    for key in parse_data:
        if parse_data[key]["published_date"] == "Error":
            continue
        else:
            dt_str = parse_data[key]['published_date']
            try:
                dt = datetime.fromisoformat(dt_str)
                formatted_date = dt.strftime("%Y-%m-%d")
                parse_data[key]["published_date"] = formatted_date
            except ValueError:
                # Handle the case where the date format is incorrect
                parse_data[key]["published_date"] = "Error"

In [45]:
# Obtener los links de las noticias (El Espectador)

parse_data = {}
sentinela = True
num = 1
link = 'https://www.elespectador.com/archivo/politica/{pag}/'
    
while sentinela == True:
    
    resultados = []
    
    driver = webdriver.Chrome()
    driver.get(link.format(pag=num))
    time.sleep(3)
        
    scroll()
    url = driver.find_element(By.XPATH, '//*[@id="main-layout-12-13"]/div[1]/div/div[1]/div/div[2]/div[2]/a')
        
    resultados.append(url.get_attribute('href'))
    
    for i in range(1,6):
        url = driver.find_element(By.XPATH, '//*[@id="main-layout-12-13"]/div[3]/div/div['+str(i)+']/div/div[2]/h2/a')
        resultados.append(url.get_attribute('href'))
    
    for noticia in resultados:
        get_news(noticia, parse_data)  
    
    num += 1
    
    last_key = list(parse_data.keys())[-1]
    dt_str = parse_data[last_key]['published_date']
    
    try: 
        dt = datetime.fromisoformat(dt_str)
        print(dt)
        
        if dt.month != 1:
            sentinela = False
            
    except:
        print("Error verificando la fecha")
        continue
        
    driver.quit()
        

2025-01-26 01:53:51.834000+00:00
2025-01-25 20:51:20.773000+00:00
2025-01-24 21:49:18.778000+00:00
2025-01-24 14:13:00.741000+00:00
2025-01-24 01:14:44.642000+00:00
2025-01-23 18:32:12.129000+00:00
2025-01-23 11:38:38.448000+00:00
2025-01-22 22:41:44.960000+00:00
2025-01-22 15:15:06.911000+00:00
2025-01-22 12:05:00+00:00
2025-01-21 23:22:55.148000+00:00
2025-01-21 18:26:58.483000+00:00
2025-01-21 11:39:52.246000+00:00
2025-01-20 23:35:28.268000+00:00
2025-01-20 18:04:42.188000+00:00
2025-01-20 13:25:30.263000+00:00
2025-01-19 22:17:45.406000+00:00
2025-01-18 19:39:47.486000+00:00
2025-01-18 14:37:26.135000+00:00
2025-01-17 21:35:05.868000+00:00
2025-01-17 12:03:00+00:00
2025-01-16 22:24:31.081000+00:00
2025-01-16 14:47:55.109000+00:00
2025-01-16 01:57:34.765000+00:00
2025-01-15 20:05:16.570000+00:00
2025-01-15 12:59:03.378000+00:00
2025-01-14 23:24:35.194000+00:00
2025-01-14 16:58:25.789000+00:00
2025-01-14 00:16:26.546000+00:00
2025-01-13 15:05:00+00:00
2025-01-12 18:56:16.672000+00:0

In [46]:
len(parse_data)

287

In [47]:
parse_data

{'“Me entristece ver a América Latina volviendo al pasado”: John Otis': {'title': '“Me entristece ver a América Latina volviendo al pasado”: John Otis',
  'published_date': '2025-01-26 15:00:00+00:00',
  'top_image': 'https://www.elespectador.com/pf/resources/images/redes_espectador.jpg?d=1011',
  'link': 'www.elespectador.com',
  'full_link': 'https://www.elespectador.com/colombia/me-entristece-ver-a-america-latina-volviendo-al-pasado-jhon-otis/',
  'text': 'John Otis nació en 1962 en Minnesota, EE. UU., vive en Colombia desde 1997 y la recorre en bicicleta. Foto: Gustavo Torrijos Zuluaga\n\nResume e infórmame rápido\n\nEscucha este artículo Audio generado con IA de Google 0:00 / 0:00 1 x\n\nJohn Otis es un periodista estadounidense que vino a Colombia, en 1997, para informar sobre la guerra como corresponsal de la revista Time y se quedó. Hoy sigue trabajando desde Bogotá para medios como la Radio Pública de los Estados Unidos y es el consultor del Comité de Protección de Periodistas

In [48]:
# Ajustamos las fechas

adjust_date(parse_data)

In [ ]:
# Convertimos el dicccionario en un DataFrame

df_extraidos = pd.DataFrame(list(parse_data.values()))